In [15]:
import sympy
import pandas as pd
from sympy import nsimplify, sin, pi, integrate
sympy.init_printing()
from sympy.printing.latex import LatexPrinter, print_latex
from io import StringIO  # Python3
import sys

# Description
This file computes the possible energy eigenvalues  and the correspoding probabilities for the problem 7.37 of the Levine, Quantum Chemistry, 7th edition.
The system is a particle in a box with the non-stationary wavefunction  


$ \Psi(x) =  \left (\frac{105}{l^7}\right)^{1/2} x^2(l-x) $    
A more detailed explanation of this procedure is in my notebook (black cover/brand: mead) for Levine solutions



In [16]:
# Quantum numbers (n) to compute
n_values = [1,2,3,4,5]



# Defining variables 
x, n, l, m, h = sympy.symbols('x n l m h')
# where
# l = length of box
# n = energy quantum number
# x = x coordinate
# m = particle mass
# h = planck constant


In [17]:
# Defining functions
A = (105/l**7)**0.5
f = x**2*(l-x)
Psi = A*f # non-stationary-state wavefunction

B = (2/l)**0.5
g = sin(n*pi*x/l)
psi = B*g # particle-in-box eigenfunction

energy_n = n**2*h**2/(8*m*l**2) # eigenvalue for a particle-in-box eigenfunction


In [18]:
# display(nsimplify(psi)) # psi function without decimals in the constant. Sympy does not show the original form

The coefficients are computed by projecting the eigenfunction $\psi_n$ from the left of the non stationary wavefunction, $\Psi$. 
The projection is done by integrating over the specified boundary conditions of the box.

$c_n = \big \langle \psi_n \big | \; \Psi  \big \rangle$\
$c_n = \int_{0}^{l} \psi_n(x) * \Psi(x) dx$

The probability of the state, $P_n$, is given by the square of the coefficient

$P_n = |c_n|^2$




In [19]:
# Integrating function or projecting coefficient
integrad = psi*Psi
coefficient_n = integrate(integrad, (x, 0, l))
probability_n = coefficient_n**2

In [20]:
def get_latex_string(expresion, mathmode = True):
    
    ## This part redirects the standard output from "print_latex" function to a variable named "result_string" 
    
    old_stdout = sys.stdout # Store the reference, to show things again in standard output
    result = StringIO()     # This variable will store everything that is sent to the standard output
    sys.stdout = result
    
    print_latex(expresion)  # sympy function that prints out latex code from expresion
    
    sys.stdout = old_stdout # Redirect again the std output to screen
    result_string = result.getvalue()
    
    
    ## Formatting of the string
    
    result_string = result_string[:-1] # Discarding line jump "\n" at the end
    if 'frac' in result_string:        # 'frac' for some reason is not intepreted by a pandas table. Only 'tfrac' is
        result_string = result_string.replace('frac', 'tfrac')
        
    if mathmode == True: # allows latex code to be interpreted by pandas table in latex math mode
        result_string = "$$" + result_string + "$$" # For some reason pandas needs two dolar signs to enter math mode
        
    
    return result_string
    
    
    
# Printing values table using string ouput
raw_data = []
symbolic_data = []
n_values_indices = []
for n_i in n_values:
    energy_ni_value = energy_n.subs(n, n_i)                       # evaluating function with value n_i
    probability_ni_value = nsimplify(probability_n.subs(n, n_i))  # using nsimplify b/c sympy is not smart enough to simplify powers of l(length)
    prob_analytical_form = str(probability_ni_value)              # you need to transform the output of sympy to string, otherwise get an error in format function 
    prob_numerical_value = float((probability_ni_value).evalf())  # evaluation numerically the analytical expresion. Tranform to float, otherwise get an error in format function

    energy_string = get_latex_string(energy_ni_value, mathmode = True)
    probability_string = get_latex_string(probability_ni_value, mathmode = True)
    
    raw_data +=      [[energy_ni_value, prob_analytical_form, prob_numerical_value]] # this data does not contain any latex formatting
    #symbolic_data += [[energy_string, probability_string[:-2] + '  = $$', prob_numerical_value]]
    symbolic_data += [[energy_string, probability_string, "=    {:.5f}".format(prob_numerical_value)]]
    n_values_indices += ['n = {}'.format(n_i)]


# **Printing tables using pandas data table**


In [21]:
pd.DataFrame(symbolic_data, columns=['Energies', 'Symbolic prob', 'Numeric prob'], index=n_values_indices)

,Energies,Symbolic prob,Numeric prob
n = 1,$$\tfrac{h^{2}}{8 l^{2} m}$$,$$\tfrac{840}{\pi^{6}}$$,= 0.87374
n = 2,$$\tfrac{h^{2}}{2 l^{2} m}$$,$$\tfrac{945}{8 \pi^{6}}$$,= 0.12287
n = 3,$$\tfrac{9 h^{2}}{8 l^{2} m}$$,$$\tfrac{280}{243 \pi^{6}}$$,= 0.00120
n = 4,$$\tfrac{2 h^{2}}{l^{2} m}$$,$$\tfrac{945}{512 \pi^{6}}$$,= 0.00192
n = 5,$$\tfrac{25 h^{2}}{8 l^{2} m}$$,$$\tfrac{168}{3125 \pi^{6}}$$,= 0.00006
